# Langchain 多任务应用开发

# 学习目标

1. 如何使用 LangChain: 一套在大模型能力上封装的工具框架  
2. 如何用几行代码实现一个复杂的 AI 应用  
3. 面向大模型的流程开发的过程抽象

# 写在前面：

- LangChain 也是一套面向大模型的开发框架 (SDK)  
- LangChain 是 AGI 时代软件工程的一个探索和原型  
- 学习 LangChain 要关注接口变更

# LangChain 的核心组件

1. 模型 I/O 封装

- Chat Models: 对语言模型接口的封装  
- PromptTemple: 提示词模板  
- OutputParser: 解析输出

2. 数据连接封装（弱于 LlamalIndex）

- Document Loaders: 各种格式文件的加载器  
- Document Transformers: 对文档的常用操作，如：split, filter, translate, extract metadata, etc  
- Text Embedding Models: 文本向量化表示，用于检索等操作  
- Verctorstores & Retrievers: 向量数据库与向量检索

# 3. 架构封装

- Chain/LCEL: 实现一个功能或者一系列顺序功能组合  
- Agent: 根据用户输入,自动规划执行步骤,自动选择每步需要的工具,最终完成用户指定的功能  
- Tools: 调用外部功能的函数,例如：调 google 搜索、文件 I/O、Linux Shell 等等  
- LangGraph: 工作流开发框架  
4. LangSmith: 过程监控与调试框架
  
![](./img/1.png)

# 文档（以Python版为例）

- 功能模块：https://python.langchain.com/docs/tutorials  
- API 文档: https://python.langchain.com/api_reference/  
- 三方组件集成：https://python.langchain.com/docs/integrations/provider/  
- 更多 HowTo: https://python.langchain.com/docs/how_to/

# LangChain 是开源项目

项目地址：https://github.com/langchain-ai

# 1. 模型 I/O 封装

把不同的模型,统一封装成一个接口,方便更换模型而不用重构代码。  
## 1.1 模型 API: ChatModel  
### 1.1.1 OpenAI模型封装










In [2]:
#!pip install -U langchain
#!pip install -U langchain-openai
#!pip install openai
#!pip install langchain-deepseek

In [1]:
from langchain.chat_models import init_chat_model
model = init_chat_model('deepseek-chat',model_provider='deepseek',
        api_key= "sk-fa1e532719fc46aa83166d15619bb76a"
    )
print(model)
response = model.invoke('你是谁')
print(response.content)

profile={} client=<openai.resources.chat.completions.completions.Completions object at 0x1107e2950> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x110da6b90> root_client=<openai.OpenAI object at 0x1103e87d0> root_async_client=<openai.AsyncOpenAI object at 0x1103e9f50> model_name='deepseek-chat' model_kwargs={} openai_api_key=SecretStr('**********') api_key=SecretStr('**********') api_base='https://api.deepseek.com/v1'
你好！我是DeepSeek，由深度求索公司创造的AI助手！😊

我是一个纯文本模型，虽然不支持多模态识别功能，但我有文件上传功能，可以帮你处理图像、txt、pdf、ppt、word、excel等各种文件，从中读取文字信息进行分析处理。我完全免费使用，拥有128K的上下文长度，还支持联网搜索功能（需要你在Web/App中手动点开联网搜索按键）。

你可以通过官方应用商店下载我的App来使用我。我很乐意为你解答问题、协助处理文档、进行对话交流等等！

有什么我可以帮助你的吗？无论是学习、工作还是日常问题，我都很愿意为你提供帮助！✨


###  1.1.2 多轮对话Seesion 封装

In [2]:
from langchain_core.messages import(
    AIMessage, # 等价于 OpenAI 接口中的 assisant role
    HumanMessage, # 等价于 OpenAI 接口中的 user role
    SystemMessage # 等价于 OpenAI 接口中的 system role
)

messages = [
    SystemMessage(content="你是 lozes 大模型课程助理。"),
    HumanMessage(content="我是 lojzes 大模型学员，我叫 lojzes"),
    AIMessage(content="欢迎！"),
    HumanMessage(content="我是谁？")
]

response = model.invoke(messages)
print(response.content)

你是 lojzes，是 lojzes 大模型课程的学员。有什么我可以帮你的吗？


### 1.1.4 流式输出

In [3]:
for token in model.stream("你是谁"):
    print(token.content,end="")

你好！我是DeepSeek，由深度求索公司创造的AI助手！😊

我是一个纯文本模型，虽然不支持多模态识别功能，但我有文件上传功能，可以帮你处理图像、txt、pdf、ppt、word、excel等文件，并从中读取文字信息进行分析处理。我完全免费使用，拥有128K的上下文长度，还支持联网搜索功能（需要你在Web/App中手动点开联网搜索按键）。

你可以通过官方应用商店下载我的App来使用。我很乐意帮助你解答问题、处理文档、进行对话交流等等！

有什么我可以帮助你的吗？无论是学习、工作还是日常问题，我都很愿意为你提供支持！✨

### 集成千问 ChatOpenAI

In [4]:
from langchain_openai import ChatOpenAI

chatLLM = ChatOpenAI(
    api_key="sk-66bc27a6330f434f8751f8172a73064f",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你是谁？"}]
response = chatLLM.invoke(messages)
print(response.content)


我是通义千问（Qwen），是阿里巴巴集团旗下的通义实验室自主研发的超大规模语言模型。我可以帮助你回答问题、创作文字、提供信息查询服务，还能陪你聊天、写故事、写公文、写邮件、写剧本等。如果你有任何需要帮助的地方，欢迎随时告诉我！


### 集成千问 DashScope

In [ ]:
# pip install langchain-community
# pip install dashscope

In [5]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage

chatLLM = ChatTongyi(
    api_key="sk-66bc27a6330f434f8751f8172a73064f",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",   # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    streaming=True,
)
res = chatLLM.stream([HumanMessage(content="hi")], streaming=True)
for r in res:
    print("chat resp:", r.content)


chat resp: Hello
chat resp: ! How can
chat resp:  I assist you
chat resp:  today?
chat resp:  😊
chat resp: 
chat resp: 


###  Ollama 模型封装

In [ ]:
# pip install langchain-ollama

In [ ]:

from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:8b",
    temperature=0.7
)

messages = [
    (
        "system","你是个专业的翻译家，可以将用户的中文翻译为英文."
    ),
    (
        "human","我喜欢编程"
    )
]

ai_msg = llm.invoke(messages)
print(ai_msg.content)


## 1.2 模型的输入与输出

![](./img/2.png)

### 1.2.1 Prompt 模板封装
### 1. PromptTemplate 可以 在模板中定义变量


In [6]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template("给我讲一个关于{subject}的笑话")
print("===Template===")
print(template)
print("===Prompt===")
print(template.format(subject='小米'))


===Template===
input_variables=['subject'] input_types={} partial_variables={} template='给我讲一个关于{subject}的笑话'
===Prompt===
给我讲一个关于小米的笑话


### 2. ChatPromptTemplate 用模板表示的对话上下文

In [7]:

from langchain_core.prompts import(
    ChatPromptTemplate,HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

from langchain_openai import ChatOpenAI

chatLLM = ChatOpenAI(
    api_key="sk-66bc27a6330f434f8751f8172a73064f",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    model="qwen-plus",  # 此处以qwen-plus为例，您可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    # other params...
)

template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("你是{product}的客服助手,你的名字叫{name}"),
    HumanMessagePromptTemplate.from_template("{query}")
])


prompt = template.format_messages(
    product="lojzes大模型",
    name="小lojzes",
    query="你是谁"
)

print(prompt)

response = chatLLM.invoke(prompt)

print(response.content)


[SystemMessage(content='你是lojzes大模型的客服助手,你的名字叫小lojzes', additional_kwargs={}, response_metadata={}), HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={})]
嗨，我是小lojzes，是lojzes大模型的客服助手~很高兴认识你！我在这里可以帮你解答各种问题，陪你聊天，或者一起探索有趣的话题。有什么我可以帮你的吗？(•̀ᴗ•́)و


### 3. MessagesPlaceHolder 把多轮对话变成模板

In [8]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

human_prompt = "Translate your answer to {language}."
human_message_template= HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder("history"),human_message_template]
)

from langchain_core.messages import AIMessage,HumanMessage

human_message = HumanMessage(content="who is Elon Musk?")
ai_message = AIMessage(content="Elon Musk is a billionaire entrepreneur,inventor,and industry designer")

messages = chat_prompt.format_prompt(
     history=[human_message,ai_message],language="中文"
)

print(messages.to_messages())


result = chatLLM.invoke(messages)
print(result.content)



[HumanMessage(content='who is Elon Musk?', additional_kwargs={}, response_metadata={}), AIMessage(content='Elon Musk is a billionaire entrepreneur,inventor,and industry designer', additional_kwargs={}, response_metadata={}), HumanMessage(content='Translate your answer to 中文.', additional_kwargs={}, response_metadata={})]
埃隆·马斯克是一位亿万富翁企业家、发明家和工业设计师。


### 1.2.2 从文件加载Prompt 模板

In [9]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_file('./prompt_template.txt',encoding="utf-8")
print("====Template====")
print(template)
print("====Prompt====")
print(template.format(topic="小明"))


====Template====
input_variables=['topic'] input_types={} partial_variables={} template='举一个关于{topic}的例子'
====Prompt====
举一个关于小明的例子


## 1.3 结构化输出
### 1.3.1 直接输出Pydamic 对象

In [10]:
from pydantic import BaseModel ,Field

# 定义输出答案

class Date(BaseModel):
    year:int = Field(description="Year")
    month:int = Field(description="Month")
    day:int = Field(description="Day")
    ear:int = Field(description="BC or AD")


from langchain_core.prompts import(
PromptTemplate,HumanMessagePromptTemplate
) 

from langchain_core.output_parsers import PydanticOutputParser
from langchain.chat_models import init_chat_model


llm = init_chat_model('deepseek-chat',model_provider='deepseek',
        api_key= "sk-fa1e532719fc46aa83166d15619bb76a"
    )

# 定义结构化输出的模型

strurctured_llm = llm.with_structured_output(Date)

template = """
提取用户输入中的日期。
用户输入:
{query}
"""

prompt = PromptTemplate(
    template = template
)
query="2025年12月11日的天气晴"
input_prompt = prompt.format_prompt(query=query)
strurctured_llm.invoke(input_prompt)



Date(year=2025, month=12, day=11, ear=1)

### 1.3.2 输出指定的JSON格式

In [11]:
## OpenAI 模型的JSON格式 
## 每个模型的JSON 格式不一样
json_schema = {
    "title": "Date",
    "description": "Formated date expression",
    "type": "object",
    "properties": {
        "year": {
            "type": "integer",
            "description": "year, YYYY",
        },
        "month": {
            "type": "integer",
            "description": "month, MM",
        },
        "day": {
            "type": "integer",
            "description": "day, DD",
        },
        "era": {
            "type": "string",
            "description": "BC or AD",
        },
   },
}

strurctured_llm = llm.with_structured_output(json_schema)
strurctured_llm.invoke(input_prompt)



{'year': 2025, 'month': 12, 'day': 11, 'era': 'AD'}

### 1.3.3 使用OutputParse
OutputParser 可以按指定格式解析模型的输出

In [21]:

from langchain_core.output_parsers import JsonOutputParser


llm = init_chat_model('deepseek-chat',model_provider='deepseek',
        api_key= "sk-fa1e532719fc46aa83166d15619bb76a"
    )

parser = JsonOutputParser(pydantic_object=Date)

prompt = PromptTemplate(
    template="提取用户输入中的日期。\n用户输入:{query}\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions":parser.get_format_instructions()}
)
input_prompt = prompt.format_prompt(query=query)
output = llm.invoke(input_prompt)
print("原始输出:\n" + output.content)
print("\n 解析后：")
parser.invoke(output)

原始输出:
{"year": 2025, "month": 12, "day": 11, "ear": 1}

 解析后：


{'year': 2025, 'month': 12, 'day': 11, 'ear': 1}

也可以用 PydanticOutputParser

In [22]:
from langchain_core.output_parsers import PydanticOutputParser

llm = init_chat_model('deepseek-chat',model_provider='deepseek',
        api_key= "sk-fa1e532719fc46aa83166d15619bb76a"
    )
parser = PydanticOutputParser(pydantic_object=Date)
input_prompt = prompt.format_prompt(query=query)
output = llm.invoke(input_prompt)
print("原始输出:\n" + output.content)
print("\n 解析后：")
parser.invoke(output)


原始输出:
{"year": 2025, "month": 12, "day": 11, "ear": 1}

 解析后：


Date(year=2025, month=12, day=11, ear=1)

OutputFixingParser 利用大模型做格式自动纠错

In [23]:
from langchain_classic.output_parsers import OutputFixingParser
from langchain.chat_models import init_chat_model

# 纠错能力与大模型能力相关
new_parser = OutputFixingParser.from_llm(parser=parser,llm=llm)

bad_output = output.content.replace("11","十一")
print("PydanticOutputParser:")
try:
    parser.invoke(bad_output)
except Exception as e:
    print(e)

print("OutputFixingParser:")
new_parser.invoke(bad_output)



PydanticOutputParser:
Invalid json output: {"year": 2025, "month": 12, "day": 十一, "ear": 1}
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 
OutputFixingParser:


Date(year=2025, month=12, day=11, ear=1)

# Funcion Calling

In [24]:
from langchain_core.tools import tool  

@tool  
def add(a: int, b: int) -> int:
    """Add two integers.
     
      Args: a:First integer 
            b:Second integer I
     """
    return a + b

@tool  
def multiply(a: float, b: float) -> float:
    """Multiply two integers.

        Args: 
            a:First integer 
            b:Second integer
    """
    
    return a * b

import json
from langchain_core.messages import(
    AIMessage, # 等价于 OpenAI 接口中的 assisant role
    HumanMessage, # 等价于 OpenAI 接口中的 user role
    SystemMessage # 等价于 OpenAI 接口中的 system role
)


llm_with_tools = llm.bind_tools([add,multiply])
query = "3.5的 4 倍是多少？"
messages = [HumanMessage(query)]

output = llm_with_tools.invoke(messages)
print(json.dumps(output.tool_calls,indent=4))



[
    {
        "name": "multiply",
        "args": {
            "a": 3.5,
            "b": 4
        },
        "id": "call_00_eGQCMClIsz9soFjH4y3E4t76",
        "type": "tool_call"
    }
]


回传 Function Call 的结果

In [26]:
messages.append(output)

available_tools = {"add":add,"multiply":multiply}

for tool_call in output.tool_calls:
    selected_tool = available_tools[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

new_output = llm_with_tools.invoke(messages)
for message in messages:
    print(json.dumps(message.model_dump(),indent=4,ensure_ascii=False))
    
print(new_output.content)


{
    "content": "3.5的 4 倍是多少？",
    "additional_kwargs": {},
    "response_metadata": {},
    "type": "human",
    "name": null,
    "id": null
}
{
    "content": "我来帮您计算 3.5 的 4 倍是多少。",
    "additional_kwargs": {
        "refusal": null
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 76,
            "prompt_tokens": 402,
            "total_tokens": 478,
            "completion_tokens_details": null,
            "prompt_tokens_details": {
                "audio_tokens": null,
                "cached_tokens": 384
            },
            "prompt_cache_hit_tokens": 384,
            "prompt_cache_miss_tokens": 18
        },
        "model_provider": "deepseek",
        "model_name": "deepseek-chat",
        "system_fingerprint": "fp_eaab8d114b_prod0820_fp8_kvcache",
        "id": "ae8983f0-7e6e-40bb-ad14-f639260d5e8a",
        "finish_reason": "tool_calls",
        "logprobs": null
    },
    "type": "ai",
    "name": null,
    "id": "lc_run-

# 1.5 小结

1. LangChain 统一封装了各种模型的调用接口，包括补全型和对话型两种  
2. LangChain 提供了 PromptTemplate 类, 可以自定义带变量的模板  
3. LangChain 提供了一些列输出解析器，用于将大模型的输出解析成结构化对象  
4. LangChain 提供了 Function Calling 的封装  
5. 上述模型属于LangChain中较为实用的部分

![](./img/3.png)

![](./img/4.png)




## 2.1 文档加载器: DocumentLoaders


In [28]:
#pip install -U langchain-community pymupdf

In [29]:

from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader('./LangGraph.pdf')
pages = loader.load_and_split()
print(pages[0].page_content)


LangGraph 核心组件: Graphs, State,
Nodes, Edges
 
LangGraph 核心组件: Graphs, State, Nodes, Edges
LangGraph
主要功能
LangGraph 平台
Graph(图)
StateGraph
Compiling your graph(编译你的图)
State(状态)
Nodes(节点)
START 节点
END 节点
Edges(边)
普通边
条件边
入口点
条件入口点
 
LangGraph
 
🦜🕸 LangGraph ⚡ 以图的方式构建语言代理 ⚡
官方文档地址：https://langchain-ai.github.io/langgraph/
LangGraph 是一个用于构建有状态的多参与者应用程序的库，利用 LLM 创建代理和多代理工作流。与
其他 LLM 框架相比，它提供了以下核心优势：循环性、可控性和持久性。LangGraph 允许您定义涉及
循环的流程，这对于大多数代理架构至关重要，使其与基于 DAG 的解决方案区别开来。作为一个非常底
层的框架，它提供了对应用程序流程和状态的细粒度控制，这对于创建可靠的代理至关重要。此外，
LangGraph 包含内置的持久性，支持先进的人机协作和记忆特性。
LangGraph 的灵感来源于 Pregel 和 Apache Beam。公共接口受 NetworkX 的启发。LangGraph 是由 
LangChain Inc 开发的，它是 LangChain 的创建者，但可以在不使用 LangChain 的情况下使用。
LangGraph 平台是用于部署 LangGraph 代理的基础设施。它是一个商业解决方案，用于将代理应用程
序部署到生产环境，构建于开源的 LangGraph 框架之上。LangGraph 平台由多个组件组成，这些组件
协同工作以支持 LangGraph 应用程序的开发、部署、调试和监控：LangGraph 服务器（API）、
LangGraph SDK（API 客户端）、LangGraph CLI（构建服务器的命令行工具）、LangGraph 
Studio（用户界面/调试器）。
主要功能
 
循环和分支：在您的应用程序中实现循环和条件语句。
持久性：在图中的每个步骤之后自动保存状态。在任何时候暂停和恢复图执

## 2.2 文档处理器

### TextSplitter

In [32]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)
paragraphs = text_splitter.create_documents([pages[0].page_content])


for para in paragraphs:
  print(para.page_content)
  print('------------------------')




LangGraph 核心组件: Graphs, State,
Nodes, Edges
 
LangGraph 核心组件: Graphs, State, Nodes, Edges
LangGraph
主要功能
LangGraph 平台
Graph(图)
StateGraph
Compiling your graph(编译你的图)
State(状态)
Nodes(节点)
START 节点
END 节点
Edges(边)
普通边
条件边
入口点
条件入口点
 
LangGraph
 
🦜🕸 LangGraph ⚡ 以图的方式构建语言代理 ⚡
官方文档地址：https://langchain-ai.github.io/langgraph/
LangGraph 是一个用于构建有状态的多参与者应用程序的库，利用 LLM 创建代理和多代理工作流。与
其他 LLM 框架相比，它提供了以下核心优势：循环性、可控性和持久性。LangGraph 允许您定义涉及
循环的流程，这对于大多数代理架构至关重要，使其与基于 DAG 的解决方案区别开来。作为一个非常底
------------------------
LangGraph 是一个用于构建有状态的多参与者应用程序的库，利用 LLM 创建代理和多代理工作流。与
其他 LLM 框架相比，它提供了以下核心优势：循环性、可控性和持久性。LangGraph 允许您定义涉及
循环的流程，这对于大多数代理架构至关重要，使其与基于 DAG 的解决方案区别开来。作为一个非常底
层的框架，它提供了对应用程序流程和状态的细粒度控制，这对于创建可靠的代理至关重要。此外，
LangGraph 包含内置的持久性，支持先进的人机协作和记忆特性。
LangGraph 的灵感来源于 Pregel 和 Apache Beam。公共接口受 NetworkX 的启发。LangGraph 是由 
LangChain Inc 开发的，它是 LangChain 的创建者，但可以在不使用 LangChain 的情况下使用。
LangGraph 平台是用于部署 LangGraph 代理的基础设施。它是一个商业解决方案，用于将代理应用程
序部署到生产环境，构建于开源的 LangGraph 框架之上。LangGraph 平台由多个组件组成，这些组件
-------------------

## 2.3 向量数据库与向量检索

In [33]:
# pip install dashscope fasiss-cpu


In [37]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./LangGraph.pdf')
pages = loader.load_and_split()
# print(pages[0].page_content)


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)
texts = text_splitter.create_documents(
    [page.page_content for page in pages[:1]]
)

# 灌库

embeddings = DashScopeEmbeddings(
    model="text-embedding-v1",
    dashscope_api_key="sk-66bc27a6330f434f8751f8172a73064f"
)
index = FAISS.from_documents(texts,embeddings)
# 检索top-5 
retriver = index.as_retriever(search_kwargs={"k":2})

docs = retriver.invoke("langGraph 是什么")

for doc in docs:
    print(doc.page_content)
    print("-------")


LangGraph 核心组件: Graphs, State,
Nodes, Edges
 
LangGraph 核心组件: Graphs, State, Nodes, Edges
LangGraph
主要功能
LangGraph 平台
Graph(图)
StateGraph
Compiling your graph(编译你的图)
State(状态)
Nodes(节点)
START 节点
END 节点
Edges(边)
普通边
条件边
入口点
条件入口点
 
LangGraph
 
🦜🕸 LangGraph ⚡ 以图的方式构建语言代理 ⚡
官方文档地址：https://langchain-ai.github.io/langgraph/
LangGraph 是一个用于构建有状态的多参与者应用程序的库，利用 LLM 创建代理和多代理工作流。与
其他 LLM 框架相比，它提供了以下核心优势：循环性、可控性和持久性。LangGraph 允许您定义涉及
循环的流程，这对于大多数代理架构至关重要，使其与基于 DAG 的解决方案区别开来。作为一个非常底
-------
LangGraph 是一个用于构建有状态的多参与者应用程序的库，利用 LLM 创建代理和多代理工作流。与
其他 LLM 框架相比，它提供了以下核心优势：循环性、可控性和持久性。LangGraph 允许您定义涉及
循环的流程，这对于大多数代理架构至关重要，使其与基于 DAG 的解决方案区别开来。作为一个非常底
层的框架，它提供了对应用程序流程和状态的细粒度控制，这对于创建可靠的代理至关重要。此外，
LangGraph 包含内置的持久性，支持先进的人机协作和记忆特性。
LangGraph 的灵感来源于 Pregel 和 Apache Beam。公共接口受 NetworkX 的启发。LangGraph 是由 
LangChain Inc 开发的，它是 LangChain 的创建者，但可以在不使用 LangChain 的情况下使用。
LangGraph 平台是用于部署 LangGraph 代理的基础设施。它是一个商业解决方案，用于将代理应用程
序部署到生产环境，构建于开源的 LangGraph 框架之上。LangGraph 平台由多个组件组成，这些组件
-------


# 3. Chain 和 LangChain Expression Language (LCEL)

LangChain Expression Language（LCEL）是一种声明式语言，可轻松组合不同的调用顺序构成Chain。LCEL自创立之初就被设计为能够支持将原型投入生产环境，无需代码更改，从最简单的“提示+LLM”链到最复杂的链（已有用户成功在生产环境中运行包含数百个步骤的LCEL Chain）。

# LCEL 的一些亮点包括：

1. 流支持：使用 LCEL 构建 Chain 时，你可以获得最佳的首个令牌时间（即从输出开始到首批输出生成的时间）。对于某些 Chain，这意味着可以直接从 LLM 流式传输令牌到流输出解析器，从而以与 LLM 提供商输出原始令牌相同的速率获得解析后的、增量的输出。  
2. 异步支持：任何使用 LCEL 构建的链条都可以通过同步 API（例如，在 Jupyter 笔记本中进行原型设计时）和异步 API（例如，在 LangServe 服务器中）调用。这使得相同的代码可用于原型设计和生产环境，具有出色的性能，并能够在同一服务器中处理多个并发请求。  
3. 优化的并行执行：当你的 LCEL 链条有可以并行执行的步骤时（例如，从多个检索器中获取文档），我们会自动执行，无论是在同步还是异步接口中，以实现最小的延迟。
4. 重试和回退：为LCEL链的任何部分配置重试和回退。这是使链在规模上更可靠的绝佳方式。目前我们正在添加重试/回退的流媒体支持，因此你可以在不增加任何延迟成本的情况下获得增加的可靠性。  
5. 访问中间结果：对于更复杂的链条，访问在最终输出产生之前的中间步骤的结果通常非常有用。这可以用于让最终用户知道正在发生一些事情，甚至仅用于调试链条。你可以流式传输中间结果，并且在每个 LangServe 服务器上都可用。  
6. 输入和输出模式：输入和输出模式为每个 LCEL 链提供了从链的结构推断出的 Pydantic 和 JSONSchema 模式。这可以用于输入和输出的验证，是 LangServe 的一个组成部分。  
7. 无缝 LangSmith 跟踪集成：随着链条变得越来越复杂，理解每一步发生了什么变得越来越重要。通过 LCEL，所有步骤都自动记录到 LangSmith，以实现最大的可观察性和可调试性。  
8. 无缝 LangServe 部署集成：任何使用 LCEL 创建的链都可以轻松地使用 LangServe 进行部署。

## 3.1 Pipeline 式调用 PromptTemplate ,LLM 和OutputParser



In [41]:

from langchain_classic.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.chat_models import init_chat_model
from pydantic import BaseModel,Field
from typing import List ,Dict,Optional
from enum import Enum

# 结构化输出
class SortEnum(str,Enum):
    data = 'data'
    price = 'price'

class OrderingEnum(str,Enum):
    ascend = 'ascend'
    descend = 'descend'

class Semantics(BaseModel):
    name: Optional[str] = Field(description="流浪包名称",default=None)
    price_lower: Optional[int] = Field(description="价格下限",default=None)
    price_upper: Optional[int] = Field(description="价格上限",default=None)
    data_lower: Optional[int] = Field(description="流量下限",default=None)
    data_upper: Optional[int] = Field(description="流量上限",default=None)
    sort_by: Optional[SortEnum] = Field(description="按价格或流量排序",default=None)
    ordering: Optional[OrderingEnum] = Field(description="升序或降序排列",default=None)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是个语义解析器，你的任务是将用户的输入解析成 JSON表示。不要回答用户的问题"),
        ("human","{text}")
        
    ]
)


llm = init_chat_model('deepseek-chat',model_provider='deepseek',
        api_key= "sk-fa1e532719fc46aa83166d15619bb76a"
    )

strurctured_llm = llm.with_structured_output(Semantics)

# LCEL 表达式
runnable = (
    {
        "text":RunnablePassthrough()
    } | prompt | strurctured_llm
)

# 运行
res = runnable.invoke("不超过 100元的流量大的套餐有哪些？")

print(
    json.dumps(
        res.model_dump(),indent=4,ensure_ascii=False
    )
)


{
    "name": null,
    "price_lower": null,
    "price_upper": 100,
    "data_lower": null,
    "data_upper": null,
    "sort_by": "data",
    "ordering": "descend"
}


## 3.2 用 LCEL 实现RAG

In [2]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.prompts import ChatPromptTemplate
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_classic.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

loader = PyMuPDFLoader('./LangGraph.pdf')
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)
texts = text_splitter.create_documents(
    [page.page_content for page in pages[:1]]
)
llm = init_chat_model('deepseek-chat',model_provider='deepseek',
        api_key= "sk-fa1e532719fc46aa83166d15619bb76a"
    )
# 灌库
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1",
    dashscope_api_key="sk-66bc27a6330f434f8751f8172a73064f"
)
index = FAISS.from_documents(texts,embeddings)
# 检索top-5
retriver = index.as_retriever(search_kwargs={"k":2})

template = """ Answer the question base only on the following context:
{context}

Question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)
#Chain
rag_chain = (
    {"question":RunnablePassthrough(),"context":retriver}
    | prompt
    | llm
    | StrOutputParser()
)

# rag_chain.invoke("LangGraph 是什么？")
# 流式输出
resp = rag_chain.stream("LangGraph 是什么？")
for r in resp:
    print(r,end='',flush=True)


根据提供的上下文，LangGraph 是一个用于构建有状态的多参与者应用程序的库，它利用 LLM（大语言模型）来创建代理和多代理工作流。